In [3]:
import pandas as pd

abs_path = "./labeled/50_abs_jose_ramos_20240308.csv"
pitches_path = "./labeled/95_pitches_jose_ramos_20240308.csv"

abs_df = pd.read_csv(abs_path)
pitches_df = pd.read_csv(pitches_path)

abs_df.head()

,abs_id,file_name,file_name_date,file_name_player_name,file_name_pitcher_type,file_name_at_bat_number,file_name_result,is_filename_result_consistent,inning,at_bat_result,weather,game_time
0,1,2022-09-10 Jose Ramos VS 右投 打席2_二飛.mp4,2022-09-10,Jose Ramos,RHP,2,二飛,True,4,4F,sunny,day
1,2,2022-09-01 Jose Ramos VS 左投 打席1_三振.mp4,2022-09-01,Jose Ramos,LHP,1,三振,True,1,SK,sunny,day
2,3,2022-08-19 Jose Ramos VS 右投 打席3_游滾.mp4,2022-08-19,Jose Ramos,RHP,3,游滾,True,5,6G,unknown,unknown
3,4,2022-08-23 Jose Ramos VS 左投 打席1_左全.mp4,2022-08-23,Jose Ramos,LHP,1,左全,True,1,7_HR,sunny,day
4,5,2022-08-10 Jose Ramos VS 右投 打席4_三振.mp4,2022-08-10,Jose Ramos,RHP,4,三振,True,8,CK,sunny,night


In [4]:
pitches_df.head()

,id,file_name,file_name_date,file_name_player_name,file_name_pitcher_type,file_name_at_bat_number,file_name_result,inning,abs_id,nth_pitch_ab,...,score_diffrence,is_team_leading,weather,game_time,has_swing_intention,horizontal_ending,vertical_ending,is_swing_delayed,is_swing_early,is_whiff
0,4e1e4436-2ba9-41a0-b88a-5cc4641edcbe,2022-09-10 Jose Ramos VS 右投 打席2_二飛.mp4,2022-09-10,Jose Ramos,RHP,2.0,二飛,4.0,1,1,...,1,True,sunny,day,False,outside,low,False,False,False
1,071fab8b-c044-4d30-80ee-b60f096fe0cd,2022-09-10 Jose Ramos VS 右投 打席2_二飛.mp4,2022-09-10,Jose Ramos,RHP,2.0,二飛,4.0,1,2,...,1,True,sunny,day,True,middle,high,False,False,False
2,bc4092b0-4083-4fd7-bf98-3863f7a4c82b,2022-09-01 Jose Ramos VS 左投 打席1_三振.mp4,2022-09-01,Jose Ramos,LHP,1.0,三振,1.0,2,1,...,1,True,sunny,day,True,outside,high,True,False,True
3,382d7a62-2423-4e1f-8b51-f7040c1e8e0a,2022-09-01 Jose Ramos VS 左投 打席1_三振.mp4,2022-09-01,Jose Ramos,LHP,1.0,三振,1.0,2,2,...,1,True,sunny,day,True,middle,middle,False,False,False
4,04aed05c-1302-4121-bcc2-be9a8ef1a3c4,2022-09-01 Jose Ramos VS 左投 打席1_三振.mp4,2022-09-01,Jose Ramos,LHP,1.0,三振,1.0,2,3,...,1,True,sunny,day,True,middle,high,True,False,True


In [5]:
pitches_df.columns


Index(['id', 'file_name', 'file_name_date', 'file_name_player_name',
       'file_name_pitcher_type', 'file_name_at_bat_number', 'file_name_result',
       'inning', 'abs_id', 'nth_pitch_ab', 'out_count', 'strike_count',
       'ball_count', 'is_rhp', 'at_bat_result',
       'is_filename_result_consistent', 'has_stealing_attempt', 'pitch_type',
       'pitch_release', 'is_obvious_off_zone', 'is_pitch_ended_catcher_want',
       'has_swing', 'has_first_base_runner', 'has_second_base_runner',
       'has_third_base_runner', 'has_visible_shift', 'score_diffrence',
       'is_team_leading', 'weather', 'game_time', 'has_swing_intention',
       'horizontal_ending', 'vertical_ending', 'is_swing_delayed',
       'is_swing_early', 'is_whiff'],
      dtype='object')

In [6]:
# 刪除無用欄位
columns_to_drop = [
    'file_name', 'file_name_date', 'file_name_player_name',
    'file_name_pitcher_type', 'file_name_at_bat_number', 'file_name_result',
    'inning', 'is_filename_result_consistent'
]


pitches_df_cleaned = pitches_df.drop(columns=columns_to_drop, axis=1)
pitches_df_cleaned.head()


,id,abs_id,nth_pitch_ab,out_count,strike_count,ball_count,is_rhp,at_bat_result,has_stealing_attempt,pitch_type,...,score_diffrence,is_team_leading,weather,game_time,has_swing_intention,horizontal_ending,vertical_ending,is_swing_delayed,is_swing_early,is_whiff
0,4e1e4436-2ba9-41a0-b88a-5cc4641edcbe,1,1,0,0,0,True,4F,False,slider,...,1,True,sunny,day,False,outside,low,False,False,False
1,071fab8b-c044-4d30-80ee-b60f096fe0cd,1,2,0,0,1,True,4F,False,four_seam,...,1,True,sunny,day,True,middle,high,False,False,False
2,bc4092b0-4083-4fd7-bf98-3863f7a4c82b,2,1,1,0,0,False,SK,False,four_seam,...,1,True,sunny,day,True,outside,high,True,False,True
3,382d7a62-2423-4e1f-8b51-f7040c1e8e0a,2,2,1,1,0,False,SK,False,four_seam,...,1,True,sunny,day,True,middle,middle,False,False,False
4,04aed05c-1302-4121-bcc2-be9a8ef1a3c4,2,3,1,2,0,False,SK,False,four_seam,...,1,True,sunny,day,True,middle,high,True,False,True


In [10]:
pitches_df_cleaned.columns


Index(['id', 'abs_id', 'nth_pitch_ab', 'out_count', 'strike_count',
       'ball_count', 'is_rhp', 'at_bat_result', 'has_stealing_attempt',
       'pitch_type', 'pitch_release', 'is_obvious_off_zone',
       'is_pitch_ended_catcher_want', 'has_swing', 'has_first_base_runner',
       'has_second_base_runner', 'has_third_base_runner', 'has_visible_shift',
       'score_diffrence', 'is_team_leading', 'weather', 'game_time',
       'has_swing_intention', 'horizontal_ending', 'vertical_ending',
       'is_swing_delayed', 'is_swing_early', 'is_whiff'],
      dtype='object')

### 壘上有人較積極?
出棒意圖，比壘上沒人的時候還要多

In [12]:
# 任一壘包有人
pitches_df_cleaned['any_on_base'] = pitches_df_cleaned[['has_first_base_runner', 'has_second_base_runner', 'has_third_base_runner']].any(axis=1)

swing_intention_rates = pitches_df_cleaned.groupby('any_on_base')['has_swing_intention'].mean()

swing_intention_rates

any_on_base
False    0.509804
True     0.666667
Name: has_swing_intention, dtype: float64

In [13]:
# 定義八種壘情況
def label_bases(row):
    return f"{int(row['has_first_base_runner'])}{int(row['has_second_base_runner'])}{int(row['has_third_base_runner'])}"

pitches_df_cleaned['base_combination'] = pitches_df_cleaned.apply(label_bases, axis=1)

swing_intention_rates_by_combination = pitches_df_cleaned.groupby('base_combination')['has_swing_intention'].mean()

print(swing_intention_rates_by_combination)


base_combination
000    0.509804
001    0.166667
010    0.666667
011    1.000000
100    0.631579
101    0.750000
110    1.000000
111    1.000000
Name: has_swing_intention, dtype: float64


### 拉打？